In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
from datetime import  time, datetime, timedelta
import pickle

In [2]:
# Load CSVs
location_history = pd.read_csv('Queried_Final/location_history.csv')
admission_details = pd.read_csv('Queried_Final/admission_details.csv')

In [3]:
def remove_hospice_patients(admission_details, location_history):
    hospice_patients = admission_details.loc[admission_details['DISCHARGE'].str.contains('Hospice', case=False)]['PAT_ENC_CSN_ID'].tolist()
    
    # Filter out hospice patients from second_df
    filtered_location_history = location_history[~location_history['PAT_ENC_CSN_ID'].isin(hospice_patients)]
    
    return filtered_location_history, hospice_patients

# Sets the day and time the first night on unit should begin based on the time of arrival at a location
def set_first_night(time):
    # If the patient arrives after 11pm, the first full night begins 11pm the following day
    if (time.hour == 23 and time.minute > 0):
        return time.replace(hour=23, minute=0) + timedelta(days=1)
    else:
        # If the patient arrives after midnight but before 11, the first full night begin at 11pm that day
        return time.replace(hour=23, minute=0)

# Sets the day and time the last night on unit should end based on the time of arrival at a location
def set_final_night(time):
    # If the patient leaves before 6am, the last full night ended 6am the previous day
    if(time.hour < 6):
        return time.replace(hour=6, minute=0) - timedelta(days=1)
    else:
        # If the patient leaves after 6 am, the last full night was 6am that morning 
        return time.replace(hour=6, minute=0)

# Adds columns about how long the patient stayed at a given location
def add_relevant_columns(df):
    df['START_TIME'] = pd.to_datetime(df['START_TIME'])
    df['END_TIME'] = pd.to_datetime(df['END_TIME'])

    # Apply the function to create the new column
    df['FIRST_NIGHT'] = df['START_TIME'].apply(set_first_night)
    df['FINAL_NIGHT'] = df['END_TIME'].apply(set_final_night)
    
    df['TIME_ON_UNIT'] = df['FINAL_NIGHT'] - df['FIRST_NIGHT']
    df['TIME_ON_UNIT'] = pd.to_timedelta(df['TIME_ON_UNIT'])

    df['NIGHTS_ON_UNIT'] = df['TIME_ON_UNIT'].dt.days + 1
    
    return df

# Returns false for a rows that have multiple nights on non-51600 units and for rows that start and end during the same day
def filter_rows(row):
    eleven_pm = pd.to_datetime('23:00:00').time()
    six_am = pd.to_datetime('06:00:00').time()
    
    if (row['NIGHTS_ON_UNIT'] > 0 ) :
        if ('516' in  row['RECORD_NAME']):
            return True
        else:
            return False
    if ((row['NIGHTS_ON_UNIT'] == 0) & ((row['START_TIME'].time() >= six_am) & (row['END_TIME'].time() <= eleven_pm))):
        if (row['START_TIME'].date() == row['END_TIME'].date()):
            return False
    return True

# Applies the filter rows function to each row, added a T/F column to the dataframe
def add_tf_column(df):
    tf_list = []
    
    for index, row in df.iterrows():
        tf_list.append(filter_rows(row))
    df['TRUE_FALSE'] = tf_list
    
    return df

# Drops all false rows as they do not meet the definition of patient nights
def keep_true_only(df):
    df = df[df['TRUE_FALSE']==True]
    
    return df

# Filters out all rows that occur before the patient's first stay on 51600 and after the last
def filter_outside_units(df):
    # Find the index of the first occurrence of '516' anywhere in 'RECORD_NAME'
    first_516_index = df[df['RECORD_NAME'].str.contains('516')].index.min()

    # Find the index of the last occurrence of '516' anywhere in 'RECORD_NAME'
    last_516_index = df[df['RECORD_NAME'].str.contains('516')].index.max()

    # Keep only the rows between the first and last occurrence of '516' (inclusive)
    filtered_df = df.loc[first_516_index:last_516_index]
    
    return filtered_df

# Create a function that runs the functions in order
def run_data_processing_pipeline(df):
    # Step 1: Add relevant columns
    df = add_relevant_columns(df)   
    
    # Step 2: Add a True/False column
    df = add_tf_column(df)
    
    # Step 3: Keep rows where TRUE_FALSE is True
    df = keep_true_only(df)
    
    # Step 4: Filter rows outside units
    df = filter_outside_units(df)
    
    return df

In [4]:
def format_start_end(start_time, end_time):
    if start_time.time() < time(23, 59, 59, 999) and start_time.time() > time(6, 0, 0):
        start_date = start_time.date()
        start_datetime = datetime.combine(start_date, time(23,0,0))
    else:
        start_date = start_time.date()
        start_date -= timedelta(days=1)
        start_datetime = datetime.combine(start_date, time(23,0,0))
    
    if end_time.time() > time(0, 0, 0) and end_time.time() < time(23, 0, 0):
        end_date = end_time.date()
        end_datetime = datetime.combine(end_date, time(6, 0, 0))
    else:
        end_date = end_time.date()
        end_date += timedelta(days=1)
        end_datetime = datetime.combine(end_date, time(6,0,0))
    
    return start_datetime, end_datetime

# Takes start_datetime and end_datetime and returns a list of nights between these two datetime ranges
def get_nights_between_datetimes(start_datetime, end_datetime):
    nights=[]
    current_datetime = start_datetime
    while current_datetime < end_datetime:
        nights.append([current_datetime, current_datetime+timedelta(hours=7)])
        current_datetime+= timedelta(days=1)
    return nights

# Extracts patient nights and creates a new DataFrame with columns for patient IDs, night start times, and night end times.
def process_data_to_dataframes(data_frame):
    
    only_516 = data_frame[data_frame['RECORD_NAME'].str.contains('516')]
    off_unit = data_frame[~data_frame['RECORD_NAME'].str.contains('516')]
    
    all_pat_nights = []
    all_pat_ids = []
    night_start_times = []
    night_end_times = []
    last_pat_id = None
    
    for index, row in data_frame.iterrows():
        if not last_pat_id:
            last_pat_id = row["PAT_ENC_CSN_ID"]
        if row["PAT_ENC_CSN_ID"] != last_pat_id:
            for night in all_pat_nights:
                all_pat_ids.append(last_pat_id)
                night_start_times.append(night[0])
                night_end_times.append(night[1])
            all_pat_nights = []
        start_datetime, end_datetime = format_start_end(row['START_TIME'], row['END_TIME'])
        nights = get_nights_between_datetimes(start_datetime, end_datetime)
        for night in nights:
            if night not in all_pat_nights:
                all_pat_nights.append(night)

        last_pat_id = row["PAT_ENC_CSN_ID"]
        
    for night in all_pat_nights:
        all_pat_ids.append(last_pat_id)
        night_start_times.append(night[0])
        night_end_times.append(night[1])
    
    df_dict = {"PAT_ENC_CSN_ID": all_pat_ids, "NIGHT_START": night_start_times, "NIGHT_END": night_end_times}
    pat_nights_df = pd.DataFrame(df_dict)
    
    return pat_nights_df, only_516,  off_unit

# Checks if a given time range defined by start_time and end_time is within the time range of a DataFrame row
def is_datetime_in_range(start_time, end_time, row):
    return (row['START_TIME'] <= start_time <= row['END_TIME']) & (row['START_TIME'] <= end_time <= row['END_TIME'])

# Takes two DataFrames, all_patient_nights and all_516_rows, and removes incomplete nights (nights that do not have both start and end times covered by the data).
def remove_incomplete_nights(all_patient_nights, all_516_rows):
    results = []
    for _, row in all_patient_nights.iterrows():
        patient_night_id = row['PAT_ENC_CSN_ID']
        night_start = row['NIGHT_START']
        night_end = row['NIGHT_END']
        patient_rows_from_all_rows = all_516_rows[all_516_rows['PAT_ENC_CSN_ID'] == patient_night_id]
        start_covered = False
        end_covered = False
        for __, row_516 in patient_rows_from_all_rows.iterrows():
            if (night_start > row_516['START_TIME']) and (night_start < row_516['END_TIME']):
                start_covered = True
            if (night_end > row_516['START_TIME']) and (night_end < row_516['END_TIME']):
                end_covered = True
            
            if start_covered and end_covered:
                results.append([patient_night_id, night_start, night_end])
                break  # Exit the loop when the night is covered
    df = pd.DataFrame(results, columns=['PAT_ENC_CSN_ID', 'NIGHT_START', 'NIGHT_END'])
    return df

#  Filters a DataFrame off_unit based on patient nights defined in full_df, ensuring that only data within each patient's night is included in the result.
def get_off_unit_interruptions(full_df, off_unit):
    
    # Initialize an empty DataFrame to store filtered results
    filtered_df = pd.DataFrame(columns=off_unit.columns)

    # Iterate through final_results_df and filter off_unit
    for _, row in full_df.iterrows():
        start = row['NIGHT_START']
        end = row['NIGHT_END']  
        patient_id = row['PAT_ENC_CSN_ID']
        # Filter off_unit for each patient and append to the filtered_df
        patient_off_unit = off_unit[off_unit['PAT_ENC_CSN_ID'] == patient_id]
        patient_off_unit = patient_off_unit[patient_off_unit['START_TIME'] >= start]
        patient_off_unit = patient_off_unit[patient_off_unit['END_TIME'] <= end]
        filtered_df = pd.concat([filtered_df, patient_off_unit], ignore_index=True)
    
    filtered_df['TIME_OFF_UNIT'] = filtered_df['END_TIME'] - filtered_df['START_TIME']
    
    # Display the filtered DataFrame
    return filtered_df

def get_on_unit_interruptions(only_516):
    result = []

    # Sort the DataFrame by PAT_ENC_CSN_ID and START_TIME
    #sorted_df = only_516.sort_values(by=['PAT_ENC_CSN_ID', 'START_TIME'])

    # Iterate through the DataFrame
    for i in range(1, len(only_516)):
        prev_row = only_516.iloc[i - 1]
        curr_row = only_516.iloc[i]
        
        if (
            prev_row['PAT_ENC_CSN_ID'] == curr_row['PAT_ENC_CSN_ID'] and
            prev_row['END_TIME'] == curr_row['START_TIME'] and
            prev_row['RECORD_NAME'] != curr_row['RECORD_NAME']
        ):
            result.append([
                prev_row['PAT_ENC_CSN_ID'],
                prev_row['END_TIME'],
                prev_row['RECORD_NAME'],
                curr_row['RECORD_NAME']
            ])

    # Create a DataFrame from the result list
    interruptions_on_516 = pd.DataFrame(result, columns=['PAT_ENC_CSN_ID', 'TIME', 'RECORD_NAME_1', 'RECORD_NAME_2'])

    # Print the result DataFrame
    return interruptions_on_516

def filter_on_unit_interruptions(interruptions_on_516, full_pat_nights):
    # Merge the DataFrames on 'PAT_ENC_CSN_ID'
    merged_df = pd.merge(interruptions_on_516, full_pat_nights, on='PAT_ENC_CSN_ID')

    # Filter rows where 'TIME' is between 'START' and 'END'
    filtered_df = merged_df[(merged_df['TIME'] >= merged_df['NIGHT_START']) & (merged_df['TIME'] <= merged_df['NIGHT_END'])]

    # Drop the 'START' and 'END' columns from the filtered DataFrame if needed
    filtered_df.drop(columns=['NIGHT_START', 'NIGHT_END'], inplace=True)
    
    filtered_df = filtered_df.drop_duplicates()

    return filtered_df

In [6]:
# Remove patients who were dischraged to hospice
location_history, hospice_patients = remove_hospice_patients(admission_details,location_history)
location_history['PAT_ENC_CSN_ID'].nunique()

In [8]:
filtered_df = run_data_processing_pipeline(location_history)
filtered_df

In [9]:
pat_nights, only_516, off_unit = process_data_to_dataframes(filtered_df)
full_pat_nights = remove_incomplete_nights(pat_nights,only_516)
full_pat_nights = full_pat_nights.drop_duplicates(subset=['PAT_ENC_CSN_ID', 'NIGHT_START', 'NIGHT_END'])
full_pat_nights

In [10]:
full_pat_nights['PAT_ENC_CSN_ID'].nunique()

In [11]:
# Get on and off unit interruptions
only_516 = only_516.sort_values(by=['PAT_ENC_CSN_ID', 'START_TIME'])
on_unit_interruptions = get_on_unit_interruptions(only_516)
filtered_on_unit_interruptions = filter_on_unit_interruptions(on_unit_interruptions, full_pat_nights)

off_unit_interruptions = get_off_unit_interruptions(full_pat_nights, off_unit)
off_unit_interruptions

C:\Users\lpink2\Anaconda3\envs\sleep_disturbances\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
filtered_on_unit_interruptions.to_csv('Queried_Final/on_unit_interruptions.csv')
off_unit_interruptions.to_csv('Queried_Final/off_unit_interruptions.csv')
full_pat_nights.to_csv('Queried_Final/patient_night_population.csv')

In [19]:
with open('pickled_dataframes/on_unit_interruptions_df', 'wb') as f:
    pickle.dump(filtered_on_unit_interruptions, f)
    
with open('pickled_dataframes/off_unit_interruptions_df', 'wb') as f:
    pickle.dump(off_unit_interruptions, f)

with open('pickled_dataframes/patient_night_population_df', 'wb') as f:
    pickle.dump(full_pat_nights, f)